In [11]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os

In [1]:
import torch
torch.__version__

'2.1.2+cu121'

In [ ]:
from typing import Optional, List, Tuple

from huggingface_hub import snapshot_download

from vllm import EngineArgs, LLMEngine, SamplingParams, RequestOutput
from vllm.lora.request import LoRARequest
def initialize_engine() -> LLMEngine:
    """Initialize the LLMEngine."""
    # max_loras: controls the number of LoRAs that can be used in the same
    #   batch. Larger numbers will cause higher memory usage, as each LoRA
    #   slot requires its own preallocated tensor.
    # max_lora_rank: controls the maximum supported rank of all LoRAs. Larger
    #   numbers will cause higher memory usage. If you know that all LoRAs will
    #   use the same rank, it is recommended to set this as low as possible.
    # max_cpu_loras: controls the size of the CPU LoRA cache.
    engine_args = EngineArgs(model="Mistral-7B-Instruct-v0.2",
                             enable_lora=True,
                             max_loras=32,
                             max_lora_rank=64,
                             max_cpu_loras=32,
                             max_num_seqs=256,enforce_eager=True,tensor_parallel_size=8,max_model_len=8192)
    return LLMEngine.from_engine_args(engine_args)
model = initialize_engine()

In [7]:
MoE_list_update_top_2

,pos,neg,expert,query,expert_predict,domain,cross-dataset,cross-task
0,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",[],Mistral|amazon_google-MoE-CT,You are an expert in detecting if two text des...,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",Mistral|amazon_google-MoE-CT,"['Mistral|semi_text_w-MoE-CT', 'Mistral|SimTab...","['Mistral|SimTab-MoE-CT', 'Mistral|walmart-MoE..."
1,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",[],Mistral|amazon_google-MoE-CT,You are an expert in detecting if two text des...,"['Mistral|amazon_google-MoE-CT', 'Mistral|SimT...",Mistral|amazon_google-MoE-CT,"['Mistral|SimTab-MoE-CT', 'Mistral|semi_text_w...","['Mistral|SimTab-MoE-CT', 'Mistral|walmart-MoE..."
2,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",[],Mistral|amazon_google-MoE-CT,You are an expert in detecting if two text des...,"['Mistral|amazon_google-MoE-CT', 'Mistral|walm...",Mistral|amazon_google-MoE-CT,"['Mistral|walmart-MoE-CT', 'Mistral|semi_text_...","['Mistral|walmart-MoE-CT', 'Mistral|SimTab-MoE..."
3,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",[],Mistral|amazon_google-MoE-CT,You are an expert in detecting if two text des...,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",Mistral|amazon_google-MoE-CT,"['Mistral|semi_text_w-MoE-CT', 'Mistral|SimTab...","['Mistral|SimTab-MoE-CT', 'Mistral|walmart-MoE..."
4,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",[],Mistral|amazon_google-MoE-CT,You are an expert in detecting if two text des...,"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",Mistral|amazon_google-MoE-CT,"['Mistral|semi_text_w-MoE-CT', 'Mistral|SimTab...","['Mistral|SimTab-MoE-CT', 'Mistral|walmart-MoE..."
...,...,...,...,...,...,...,...,...
29582,"['Mistral|semi_text_w-MoE-CT', 'Mistral|webtab...","['Mistral|amazon_google-MoE-CT', 'Mistral|rest...",Mistral|amazon_google-MoE-CT,You are an expert in relation extraction from ...,"['Mistral|SimTab-MoE-CT', 'Mistral|webtable-Mo...",Mistral|SimTab-MoE-CT,"['Mistral|webtable-MoE-CT', 'Mistral|restauran...","['Mistral|restaurant-MoE-CT', 'Mistral|semi_te..."
29583,"['Mistral|semi_text_w-MoE-CT', 'Mistral|webtab...","['Mistral|amazon_google-MoE-CT', 'Mistral|SimT...",Mistral|amazon_google-MoE-CT,You are an expert in relation extraction from ...,"['Mistral|SimTab-MoE-CT', 'Mistral|webtable-Mo...",Mistral|SimTab-MoE-CT,"['Mistral|webtable-MoE-CT', 'Mistral|restauran...","['Mistral|restaurant-MoE-CT', 'Mistral|semi_te..."
29584,"['Mistral|amazon_google-MoE-CT', 'Mistral|webt...","['Mistral|semi_text_w-MoE-CT', 'Mistral|restau...",Mistral|amazon_google-MoE-CT,You are an expert in relation extraction from ...,"['Mistral|SimTab-MoE-CT', 'Mistral|webtable-Mo...",Mistral|SimTab-MoE-CT,"['Mistral|webtable-MoE-CT', 'Mistral|restauran...","['Mistral|restaurant-MoE-CT', 'Mistral|semi_te..."
29585,['Mistral|webtable-MoE-CT'],"['Mistral|amazon_google-MoE-CT', 'Mistral|semi...",Mistral|amazon_google-MoE-CT,You are an expert in relation extraction from ...,"['Mistral|SimTab-MoE-CT', 'Mistral|webtable-Mo...",Mistral|SimTab-MoE-CT,"['Mistral|webtable-MoE-CT', 'Mistral|restauran...","['Mistral|restaurant-MoE-CT', 'Mistral|semi_te..."


In [8]:
MoE_list_update_top_2 = pd.read_csv('Router/MoE_list_update_top_2.csv',index_col=0)
def AST(row):
    CD = row['cross-dataset']
    row['cross-dataset'] = eval(CD)
    CT = row['cross-task']
    row['cross-task'] = eval(CT)
    return row
MoE_list_update_top_2 = MoE_list_update_top_2.apply(AST,axis=1)

In [9]:
expert_list = []
for index,row in MoE_list_update_top_2.iterrows():
    expert_list.append(set(row['cross-dataset'])) if set(row['cross-dataset']) not in expert_list else None
    expert_list.append(set(row['cross-task'])) if set(row['cross-task']) not in expert_list else None

In [13]:
lora_path_dict = {}
for e in expert_list:
    expert_0,expert_1 = list(e)
    folder_path = 'lora_weight/MoE_CT/add/Mistral/merge/%s#%s' % (expert_0,expert_1)
    folder_path_rev = 'lora_weight/MoE_CT/add/Mistral/merge/%s#%s' % (expert_1,expert_0)

    if(os.path.exists(folder_path)):
        lora_path_dict['%s#%s' % (expert_0,expert_1)] = folder_path
        lora_path_dict['%s#%s' % (expert_1,expert_0)] = folder_path
    elif(os.path.exists(folder_path_rev)):
        lora_path_dict['%s#%s' % (expert_0,expert_1)] = folder_path_rev
        lora_path_dict['%s#%s' % (expert_1,expert_0)] = folder_path_rev
    else:
        print(folder_path)
    

In [72]:
# lora_path_dict = {'hospital':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/hospital-train-MoE-Llama-2-7b',
#                   'beer':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/beer-train-MoE-Llama-2-7b',
#                   'rayyan':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/rayyan-train-MoE-Llama-2-7b',
#                   'merge-linear':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/data-cleaning-adapter',
#                   'merge-cat':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/data-cleaning-adapter-cat',
#                   'merge-hospital-linear':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE/data-cleaning-adapter-hospital',
#                   'Mixtral-baseline':'/home/yanmy/LLaMA-Factory-main/lora_weight/Mixtral-baseline-MTL',
#                   'jellyfish-baseline':'/home/yanmy/LLaMA-Factory-main/lora_weight/jellyfish-baseline-MTL',
#                   'MoE-CT-amazon-google':'/home/yanmy/LLaMA-Factory-main/lora_weight/MoE_CT/add/Mistral/amazon_google-MoE-CT'}
# def create_multi_lora_call(df,lora_id_list=['']):
#     multi_lora_call = []
#     if(lora_id_list!=['']): ## Lora_ID被指定
#         # text_list = df['instruction'].to_list()
#         for lora_id in lora_id_list:
#             multi_lora_call.extend([[row['instruction'],lora_id,index] for index,row in df.iterrows()])
#     else:
#         # text_list = df['instruction'].to_list()
#         # multi_lora_call = df['instruction'].to_list()
#         # for index,row in df.iterrows():
#         multi_lora_call = [[row['instruction'],row['lora_id'],index] for index,row in df.iterrows()]
#     return multi_lora_call

def create_multi_lora_call(df,lora_id_list=['']):
    multi_lora_call = []
    for index,row in df.iterrows():
        lora_id = '#'.join(row['cross-task'])
        multi_lora_call.append([row['query'],lora_id,index])
    return multi_lora_call
def create_test_prompts(multi_lora_call: list,lora_path: dict)-> List[Tuple[str, SamplingParams]]:
    output_list = []
    lora_all = list(lora_path.keys())
    for m in multi_lora_call:
        m_output = ("[INST] %s [/INST]" % m[0],
         SamplingParams(temperature=0.0,
                        # top_p=1,
                        # prompt_logprobs=1,
                        max_tokens=32),
         LoRARequest(m[1], lora_all.index(m[1]) + 1, lora_path[m[1]]),
         m[2])
        output_list.append(m_output)
    return output_list
# multi_lora_call = create_multi_lora_call(hospital_test,lora_id='merge-hospital-linear')
lora_id_list=list(lora_path_dict.keys())
multi_lora_call = create_multi_lora_call(MoE_list_update_top_2,lora_id_list=list(lora_path_dict.keys()))
test_prompts_input = create_test_prompts(multi_lora_call=multi_lora_call,lora_path=lora_path_dict)

In [ ]:
from datetime import datetime
def process_requests(engine: LLMEngine,
                     test_prompts: List[Tuple[str, SamplingParams,
                                              Optional[LoRARequest]]]):
    """Continuously process a list of prompts and handle the outputs."""
    request_id = 0
    output_list = []
    output_request = []
    while test_prompts or engine.has_unfinished_requests():
        
        if test_prompts:
            prompt, sampling_params, lora_request, index = test_prompts.pop(0)
            engine.add_request(str(index),
                               prompt,
                               
                               sampling_params,
                               lora_request=lora_request)
            # print(index)
            request_id += 1
        # print(request_id)
        request_outputs: List[RequestOutput] = engine.step()
        # output_request.extend(request_outputs)
        for request_output in request_outputs:
            if request_output.finished:
                output_list.append(request_output)
    # output_list.reverse()
    return output_list
lora_id_list=list(lora_path_dict.keys())
multi_lora_call = create_multi_lora_call(MoE_list_update_top_2,lora_id_list=list(lora_path_dict.keys()))
test_prompts_input = create_test_prompts(multi_lora_call=multi_lora_call,lora_path=lora_path_dict)
start_time = datetime.now()
result_all = process_requests(model, test_prompts=test_prompts_input)
end_time = datetime.now()
print((end_time-start_time).total_seconds())

In [ ]:
54.178117 Mixtral
37.780711 jellyfish

4100 For MoE Architecture base on Code
4059

## Process output

In [19]:
# print(result_all[0].prompt)
# result_all[33]
output_ins = {}
output_predict = {}
for lora_id in lora_id_list:
    # output_ins[lora_id] = [''] * int(len(result_all) / len(lora_path_dict))
    # output_predict[lora_id] = [''] * int(len(result_all) / len(lora_path_dict))
    output_ins[lora_id] = [''] * int(len(result_all) )
    output_predict[lora_id] = [''] * int(len(result_all) )
# output_lora_id = [''] * len(result_all)
for request in result_all:
    request_id = int(request.request_id)
    request_ins = request.prompt.strip()
    request_lora = request.lora_request.lora_name
    request_output = request.outputs[0].text.strip()
    output_ins[request_lora][request_id] = request_ins
    output_predict[request_lora][request_id] = request_output
    # output_lora_id[request_id] = request_lora

In [41]:
print(MoE_list_update_top_2.iloc[-1,3])

You are an expert in relation extraction from wikipedia web table to knowledge graph. Given table title and column pair for Table 1, please choose the most proper type from the provided options. Return in json format.

Column: col0

Options:["OfficeHolder", "Congressman", "Senator"]

Output Format Example:

{"type": ""}

Table 1:

{"col0": "M. N. Khan", "col1": "Malik Noor Khan", "col2": "2011-12-15", "col3": "1941"}
{"col0": "M. Marye", "col1": "Madison Ellis Marye", "col2": "2016-02-23", "col3": "1944"}
{"col0": "J. T. Lozano", "col1": "Jorge Tadeo Lozano de Peralta y González Manrique", "col2": "1816-07-06", "col3": "1772"}
{"col0": "F. Knox", "col1": "William Franklin Knox", "col2": "1944-04-28", "col3": "1898"}
{"col0": "F. Church", "col1": "Frank Forrester Church III", "col2": "1984-04-07", "col3": "1943"}

Reference tables:

{'Table': '{"col0": "American Popular Revolutionary Alliance", "col1": "Social democracy", "col2": "J. d. Castillo", "col3": "1924", "col4": "1924-05-07"}\n

In [77]:
# result_all[12000].outputs[0].text.strip()
# print(result_all[-500].prompt.strip())
# print(result_all[-500].outputs[0].text.strip())
dict_output_MoE = {}
for result in result_all:
    prompt = result.prompt.replace('[INST] ','').replace(' [/INST]','')
    output = result.outputs[0].text.strip()
    dict_output_MoE[prompt] = output

In [78]:
key_example = list(dict_output_MoE.keys())

In [79]:
MoE_list_update_top_2['prediction'] = MoE_list_update_top_2['query'].map(dict_output_MoE)

In [81]:
count = 0
cound_index = []
for index,row in MoE_list_update_top_2.iterrows():
    predict = row['prediction']
    try:
        eval(predict)
    except:
        cound_index.append(index)
        

In [82]:
len(cound_index)

3735

In [83]:
MoE_list_update_top_2.to_csv('Router/MoE_list_update_top_2_cross_task.csv')